In [1]:
%matplotlib inline
import pandas as pd
from scipy import stats
import seaborn as sns

import numpy as np
from collections import Counter

In [63]:
data=pd.read_csv("data/clean.csv")

In [64]:
df=data.copy()

In [65]:
numeric = []
qualitative= []

for variable in df.columns.tolist():
    if df.dtypes[variable] in ['float64','int64']:
        numeric.append(variable)
    else :
        qualitative.append(variable)

##### Test de normalité

In [66]:
def shapiro_test(x):
    try :
        res = stats.shapiro(x)
    except :
        return -1
    alpha = 0.05

    print("p = ",res.pvalue)

    if res.pvalue < alpha:  # null hypothesis: x comes from a normal distribution

        print("(shapiro)The null hypothesis can be rejected -> X ne possède pas une distribution normale")

    else:

        print("(shapiro) The null hypothesis cannot be rejected -> X possède éventuellement une distribution normale")

    return res

def omnibus_normaltest(x):
    #test D'Agostino-Pearson
    try :
        k2, p = stats.normaltest(x)
    except :
        return -1
    
    alpha = 0.05

    print("p = ",p)

    if p < alpha:  # null hypothesis: x comes from a normal distribution

        print("(normaltest) The null hypothesis can be rejected -> X ne possède pas une distribution normale")

    else:

        print("(normaltest) The null hypothesis cannot be rejected -> X possède éventuellement une distribution normale")

    return [k2,p]

## valeur missing

In [67]:
def missing(df):
    df_null = pd.DataFrame([df.isnull().sum(),round(100*df.isnull().sum()/ len(df), 2), df.dtypes]).transpose().reset_index()
    df_null.columns = ["variable", "valeur_NA", "Pourcentage_NA", "type"]
    df_null = df_null[df_null.valeur_NA != 0].sort_values("valeur_NA",ascending = False).reset_index(drop = True)
    return df_null
missing(df)

,variable,valeur_NA,Pourcentage_NA,type
0,division_af,14744,95.49,object
1,fonction_af,10368,67.15,object
2,ann_dipl,8727,56.52,float64
3,date_ps,8524,55.2,float64
4,direction_af,8299,53.75,object
5,motif_affect,7925,51.32,object
6,service_af,6910,44.75,object
7,job_act,3040,19.69,object
8,dt_echeance,2958,19.16,float64
9,sit_fam,320,2.07,object


In [68]:
list(data.columns[data.isnull().sum() / len(data) > 0.75])

['division_af']

In [69]:
d=data.copy()
df = d.drop('division_af',axis=1)

In [70]:
d=list(data.columns[data.isnull().sum() / len(data) < 0.05])
for i in list(d):
    df = df.dropna(subset=[i], axis=0)
missing(df)

,variable,valeur_NA,Pourcentage_NA,type
0,fonction_af,10065,67.15,object
1,ann_dipl,8399,56.03,float64
2,date_ps,8380,55.91,float64
3,direction_af,8194,54.67,object
4,motif_affect,7671,51.18,object
5,service_af,6794,45.33,object
6,job_act,2970,19.81,object
7,dt_echeance,2927,19.53,float64


In [78]:
df['fonction_af'] = df['fonction_af'].fillna(df['fonction_af'].mode()[0])
df['direction_af'] = df['direction_af'].fillna(df['direction_af'].mode()[0])
df['motif_affect'] = df['motif_affect'].fillna(df['motif_affect'].mode()[0])
df['service_af'] = df['service_af'].fillna(df['service_af'].mode()[0])
df['job_act'] = df['job_act'].fillna(df['job_act'].mode()[0])

d=list(df.columns[df.isnull().sum() / len(df) < 0.05])
for i in list(d):
    df = df.dropna(subset=[i], axis=0)
    
from sklearn.impute import KNNImputer
df[numeric]=KNNImputer(missing_values=np.nan, n_neighbors=3).fit_transform(df[numeric])

In [79]:
missing(df)

,variable,valeur_NA,Pourcentage_NA,type


## Valeur aberante

In [85]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

Outliers_to_drop = detect_outliers(df,0,numeric)
# df.drop(Outliers_to_drop,0,inplace=True)
len(Outliers_to_drop)

3135

## Teste de normalite 

In [86]:
# Shapiro-Wilk Test
from scipy.stats import shapiro
def Shapiro(d):
    stat, p = shapiro(d)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')
    
for i in numeric:
    print(i)
    Shapiro(df[i])

age
Statistics=0.975, p=0.000
Sample does not look Gaussian (reject H0)
job_ok
Statistics=0.093, p=0.000
Sample does not look Gaussian (reject H0)
nbre_enf
Statistics=0.888, p=0.000
Sample does not look Gaussian (reject H0)
ann_dipl
Statistics=0.092, p=0.000
Sample does not look Gaussian (reject H0)
ordre
Statistics=0.845, p=0.000
Sample does not look Gaussian (reject H0)
annee_ent
Statistics=0.885, p=0.000
Sample does not look Gaussian (reject H0)
date_ent
Statistics=0.882, p=0.000
Sample does not look Gaussian (reject H0)
date_ps
Statistics=0.868, p=0.000
Sample does not look Gaussian (reject H0)
dat_retraite
Statistics=0.978, p=0.000
Sample does not look Gaussian (reject H0)
dt_echeance
Statistics=0.770, p=0.000
Sample does not look Gaussian (reject H0)
tot_afct
Statistics=0.933, p=0.000
Sample does not look Gaussian (reject H0)


/home/asina/anaconda3/lib/python3.8/site-packages/scipy/stats/morestats.py:1681: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [87]:
from scipy.stats import normaltest
def agostino_test(d):
    stat, p = normaltest(d)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')
    
for i in numeric:
    agostino_test(df[i])

Statistics=658.337, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=20449.467, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=2866.628, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=39204.476, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=3761.729, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=2254.635, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=2329.172, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=5039.294, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=376.801, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=6658.081, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=2565.770, p=0.000
Sample does not look Gaussian (reject H0)


In [88]:
p_values = {}
for col in numeric:
    print("\n",col)
    Shapiro(np.log(df[col].dropna()))


 age
Statistics=0.981, p=0.000
Sample does not look Gaussian (reject H0)

 job_ok
Statistics=nan, p=1.000
Sample looks Gaussian (fail to reject H0)

 nbre_enf
Statistics=nan, p=1.000
Sample looks Gaussian (fail to reject H0)

 ann_dipl
Statistics=0.020, p=0.000
Sample does not look Gaussian (reject H0)

 ordre
Statistics=0.925, p=0.000
Sample does not look Gaussian (reject H0)

 annee_ent
Statistics=0.867, p=0.000
Sample does not look Gaussian (reject H0)

 date_ent
Statistics=0.819, p=0.000
Sample does not look Gaussian (reject H0)

 date_ps
Statistics=0.852, p=0.000
Sample does not look Gaussian (reject H0)

 dat_retraite
Statistics=0.971, p=0.000
Sample does not look Gaussian (reject H0)

 dt_echeance
Statistics=0.507, p=0.000
Sample does not look Gaussian (reject H0)

 tot_afct
Statistics=nan, p=1.000
Sample looks Gaussian (fail to reject H0)


/home/asina/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## log

## Stendardisation

In [90]:
df.to_csv(r'data/preproce.csv',index=False)